Goal: Dev ML to predict healthy state of the crusher

Table 1 with raw sensor data
- 2min sampling size
- scalar system
- Unsupervised - no column giving state,
    - need to add state of the crusher every two min - synchronize the two data sets


Table 2 with downtime events - for plant, including crusher
- IBM SAP
- many columns, but the relevant ones are:
- one column start of downtime event, another is the end, same type of time step in the sensor data.
    - Equipment column says what what eq that caused the downtime - code or tag identifying whether is was the crusher, some other comp, or plant, or unknown (txt)
    - Failed, operational, scheduled maintenance, external, understock
    - Every row is when the whole plant was out of production - dont want this yet, want to filter out these events except "failed". The removed ones are ones that left crusher in standby (not a failure)
    - 140-CR-004 --> Crusher Code
    - Filter in such a way that eq == crusher, and downtime event type == failure
    - Downtime == pause --> healthy state == pause
    - If have the eq != crusher and downtime ==fail --> crusher is pause
Before applying filter need to rename all options then apply filter then synchronization.
Need to keep crusher standby data because data is unlabeled so after cleaning we synchronize by start and end of time stamp if time is within range of first two rows therefore the label is whatever it is in the other. If we get to end and dont find a range where this timestamp belongs that means that the crusher was not failed was not standby and should add operating label. If we delete or drop all standby we overestimate the number of times the crusher was operational. Overestimating frequency of the normal condition of the crusher. sync after all filtering.

set index for timestamp of sensor data.
going to have two dates for the downtime - use pandas to set an interval databased index. pd.intervalindec(start,end, ..). The interval index will have two dates - if dfsensor.index is > dflabels.index and dfsensor.index < df.labels then that row belongs to that interval.

At the end we dont want our model to predict the standby states.

need to slice data - how much further back you slice you give engineers more time to react. Define some advanced degradation prior to failure --> maybe provides an anticipation of 12 hours.

2 states:
- identify when the crusher was operating normally and 
- when the crusher was in a failed state.

Should be trained only with data relevant to the pattern or patterns
- normal
- deviations of normal indicating a degradation up to a failure - no indication from downtime but we do have indication of when the full plant stops with root cause of failure
- failure
